In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
search_tool=DuckDuckGoSearchRun()

In [11]:
@tool
def get_weather_data(city: str) -> str: 
  """
  This function fetches the current weather data for a given city
  """

  url = f'https://api.weatherstack.com/current?access_key=327f7b9ef8880c7d5638e8c7b93a934b&query={city}'

  response=requests.get(url)

  return response.json()


In [12]:
get_weather_data.invoke({'city':'jaipur'})

{'request': {'type': 'City',
  'query': 'Jaipur, India',
  'language': 'en',
  'unit': 'm'},
 'location': {'name': 'Jaipur',
  'country': 'India',
  'region': 'Rajasthan',
  'lat': '26.917',
  'lon': '75.817',
  'timezone_id': 'Asia/Kolkata',
  'localtime': '2025-10-29 21:21',
  'localtime_epoch': 1761772860,
  'utc_offset': '5.50'},
 'current': {'observation_time': '03:51 PM',
  'temperature': 24,
  'weather_code': 143,
  'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'],
  'weather_descriptions': ['Haze'],
  'astro': {'sunrise': '06:34 AM',
   'sunset': '05:46 PM',
   'moonrise': '01:07 PM',
   'moonset': '11:50 PM',
   'moon_phase': 'First Quarter',
   'moon_illumination': 43},
  'air_quality': {'co': '458.85',
   'no2': '7.35',
   'o3': '125',
   'so2': '10.15',
   'pm2_5': '29.15',
   'pm10': '29.85',
   'us-epa-index': '2',
   'gb-defra-index': '2'},
  'wind_speed': 7,
  'wind_degree': 68,
  'wind_dir': 'ENE',
  'pressure': 101

In [ ]:
# step 1 : create llm
llm =ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [19]:
# Step 2: Pull the ReAct prompt from LangChain Hub

prompt = hub.pull("hwchase17/react")  # pulls the standard ReAct agent prompt


In [20]:
# Step 3: Create the ReAct agent manually with the pulled prompt

agent=create_react_agent(
    llm=llm, 
    tools=[search_tool, get_weather_data], 
    prompt=prompt
)

In [22]:
# Step 4: Wrap it with AgentExecutor
agent_executor=AgentExecutor(
    agent=agent, 
    tools=[search_tool, get_weather_data], 
    verbose=True
)

In [23]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Madhya Pradesh, then find it's current weather condition"})

print(response)



> Entering new AgentExecutor chain...
Action: duckduckgo_search
Action Input: capital of Madhya Pradesh6 days ago - Its capital is Bhopal . Other major cities includes Indore, Gwalior, Ujjain, Jabalpur and Sagar. Madhya Pradesh is the second largest Indian state by area and the fifth largest state by population with over 72 million residents. It borders the states of Rajasthan to the northwest, Uttar Pradesh ... 5 days ago - Bhopal (Hindi: Bhōpāl, pronounced [bʱoːpaːlᵊ] ⓘ) is the capital city of the Indian state of Madhya Pradesh and the administrative headquarters of both Bhopal district and Bhopal division. It is known as the City of Lakes, due to presence of various natural and artificial lakes near ... 2 days ago - Indore (/ɪnˈdɔːr/ ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh. The commercial capital of the state , it has been declared as the cleanest city in India 8 years in a row. 3 weeks ago - मध्य प्रदेश (अंग्

In [24]:
response['output']

'The capital of Madhya Pradesh is Bhopal. The current weather condition in Bhopal is Haze with a temperature of 22°C. The wind speed is 17 km/h from the NE, humidity is 83%, and visibility is 4 km.'